In [1]:
#pip install simple-salesforce python-decouple

In [1]:
from simple_salesforce import Salesforce
import requests
import pandas as pd
from io import StringIO
import numpy as np
import os
from datetime import datetime

In [2]:
sf=Salesforce(username='bhaskar.hindurao@atomberg.com',password='Mumbai@2024',security_token='iNY2buqCTmsVXq2yx0mWXRY2')

In [3]:
sf_instance = 'https://atombergtechnologiespvtltd2.my.salesforce.com/' #Your Salesforce Instance URL
reportId = '00O2w00000En6wbEAB' # add report id
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df1 = pd.read_csv(StringIO(download_report))

# Get today's date in YYYY-MM-DD format
today_date = datetime.today().strftime('%d-%m-%Y')

# Append today's date to the file name
csv_file_path = f'C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/ST_{today_date}.csv'
df1.to_csv(csv_file_path, index=False)
print(f"DataFrame saved to {csv_file_path}")


C:\Users\Pratik Wable\AppData\Local\Temp\ipykernel_15116\1226093586.py:7: DtypeWarning: Columns (15,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(StringIO(download_report))


DataFrame saved to C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/ST_09-04-2024.csv


In [4]:
import pandas as pd
import os
from datetime import datetime

# Get today's date in YYYY-MM-DD format
today_date = datetime.today().strftime('%d-%m-%Y')

# Load the report file with 'latin1' encoding
report = pd.read_csv(f'C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/ST_{today_date}.csv', encoding='latin1')
'''report = pd.read_csv('C:/Users/Pratik Wable/Downloads/report1712569490900.csv', encoding='latin1')'''

# Filter the 2nd 'Status' column to only include 'Completed', 'New', and 'On Hold' rows
report['Status2'] = report['Status.1'].apply(lambda x: x if x in ['Completed', 'New', 'On Hold'] else None)

# Filter the dataframe to include only rows where 'Status2' is not None
filtered_report = report.dropna(subset=['Status2'])

# Remove rows with 'WarpDrive Test ASP' in the 'ASP/Engineer Name' column
filtered_report = filtered_report[filtered_report['ASP/Engineer Name'] != 'WarpDrive Test ASP']

# Create the 'Fake OP' folder if it doesn't exist
folder_path = 'C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake OP'
os.makedirs(folder_path, exist_ok=True)

# Save the filtered data to a new CSV file named "SSS.csv" in the 'Fake OP' folder
csv_path = os.path.join(folder_path, 'SSS.csv')
filtered_report.to_csv(csv_path, index=False)

print(f"Filtered DataFrame saved to {csv_path}")


C:\Users\Pratik Wable\AppData\Local\Temp\ipykernel_15116\896786507.py:9: DtypeWarning: Columns (15,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  report = pd.read_csv(f'C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/ST_{today_date}.csv', encoding='latin1')


Filtered DataFrame saved to C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake OP\SSS.csv


In [9]:
# Load the report file with the initial processing steps
report = pd.read_csv('C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake Op/SSS.csv')

# Add 'Zone', 'State', and 'Cities' columns with blank values
report['Zone'] = ''
report['State'] = ''
report['Cities'] = ''

# Load the Zone and Pincode files
zone = pd.read_csv('C:/Users/Pratik Wable/Desktop/AT_Office/TAT_MAIN/Zone.csv')
pincode = pd.read_csv('C:/Users/Pratik Wable/Desktop/AT_Office/TAT_MAIN/Pincode.csv')

# Perform VLOOKUP-like operations to fill in 'Zone', 'State', and 'Cities' columns
for index, row in report.iterrows():
    engineer_name = row['ASP/Engineer Name']
    zip_code = row['Zip/Postal Code']
    zone_row = zone[zone['ASP/Engineer Name'] == engineer_name]
    if not zone_row.empty:
        report.at[index, 'Zone'] = zone_row.iloc[0]['Zone']
        report.at[index, 'State'] = zone_row.iloc[0]['State']
        report.at[index, 'Cities'] = zone_row.iloc[0]['City']
    else:
        pincode_row = pincode[pincode['Zip/Postal Code'] == zip_code]
        if not pincode_row.empty:
            report.at[index, 'Zone'] = pincode_row.iloc[0]['Zone']
            report.at[index, 'State'] = pincode_row.iloc[0]['State Name']
            report.at[index, 'Cities'] = pincode_row.iloc[0]['Top Cities']

# Copy 'Created Date' column data to 'C Date' column and format as '12-Mar-24'
report['C Date'] = pd.to_datetime(report['WO Created Date'], format='%d/%m/%Y').dt.strftime('%d-%b-%y')

# Copy 'End Date.1' column data to 'E Date' column and format as '12-Mar-24'
report['E Date'] = pd.to_datetime(report['WOLI End Date'].astype('str').str.split(',').str[0], format='%d/%m/%Y').dt.strftime('%d-%b-%y')

# Save the updated report file
report.to_csv('C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake OP/OP.csv', index=False)


C:\Users\Pratik Wable\AppData\Local\Temp\ipykernel_15116\2825173606.py:2: DtypeWarning: Columns (35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  report = pd.read_csv('C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake Op/SSS.csv')
C:\Users\Pratik Wable\AppData\Local\Temp\ipykernel_15116\2825173606.py:11: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  pincode = pd.read_csv('C:/Users/Pratik Wable/Desktop/AT_Office/TAT_MAIN/Pincode.csv')


In [10]:
import pandas as pd
import numpy as np
import shutil
import os
from datetime import datetime

# Load the 'ST' file with 'latin1' encoding
st_file = 'C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake OP/OP.csv'
st_data = pd.read_csv(st_file, encoding='latin1')

# Create a new DataFrame with the specified columns
ns = pd.DataFrame(columns=['Case Number', 'Work Order Line Item Number', 'Keys', 'CallCount'])

# Copy data from 'Case Number' and 'Work Order Line Item Number' columns to the new DataFrame
ns['Case Number'] = st_data['Case Number']
ns['Work Order Line Item Number'] = st_data['Work Order Line Item Number']

ns['Keys']=ns['Case Number'].astype(str) + ns['Work Order Line Item Number'].astype(str)
ns['Keys'] = pd.to_numeric(ns['Keys'], errors='coerce')

# Add a column for Serial Count
ns['CallCount'] = ns.groupby('Keys').cumcount() + 1
ns['CallCount'] = np.where(ns['CallCount'] == 1, 1, 0)

# Copy the 'Callcount' column from the new sheet to the main sheet
st_data['CallCount'] = ns['CallCount']

# Save both main and new sheet in the same Excel file
today_date = datetime.today().strftime('%d-%m-%Y')
excel_file = f'C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/CRM_DATA_AS_ON_{today_date}.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    # Write the main sheet
    st_data.to_excel(writer, sheet_name='Main', index=False)
    
    # Write the new sheet
    ns.to_excel(writer, sheet_name='New Sheet', index=False)

# Delete the 'Fake OP' folder and its contents
shutil.rmtree('C:/Users/Pratik Wable/Desktop/AT_Office/Service Trend/Fake OP')

# Display the new sheet data
ns

C:\Users\Pratik Wable\AppData\Local\Temp\ipykernel_15116\2470917159.py:9: DtypeWarning: Columns (35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  st_data = pd.read_csv(st_file, encoding='latin1')


,Case Number,Work Order Line Item Number,Keys,CallCount
0,930138,1,9301381,1
1,929386,1,9293861,1
2,927791,1,9277911,1
3,930121,5,9301215,1
4,926236,1,9262361,1
...,...,...,...,...
18989,927906,1,9279061,1
18990,927909,1,9279091,1
18991,927898,1,9278981,1
18992,927910,1,9279101,1
